This is based on this code: https://github.com/codekansas/keras-language-modeling/blob/master/keras_models.py

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from collections import defaultdict
from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop, Adagrad
from keras.utils import np_utils
import keras.layers.convolutional
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, TimeDistributedDense
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU
from Metrics import rpf1
from load_data import load_process_essays

from window_based_tagger_config import get_config
from IdGenerator import IdGenerator as idGen
from results_procesor import ResultsProcessor, __MICRO_F1__
from Rpfa import micro_rpfa

import Settings
import logging

import datetime

Using TensorFlow backend.


In [29]:
#Helper Functions
def collapse_results(seq_lens, probs):
    assert len(seq_lens) == probs.shape[0], "Axis 1 size does not align"
    ixs = np.argmax(probs, axis=2)
    pred_ys_by_tag = defaultdict(list)
    for i in range(len(seq_lens)):
        row_ixs = ixs[i,:]
        len_of_sequence = seq_lens[i] + 2
        pred_ys = [ix2tag[j] for j in row_ixs[-len_of_sequence:]]   
        # skip the start and end label
        pred_ys = pred_ys[1:-1]
        for pred_tag in pred_ys:
            pred_ys_by_tag[pred_tag].append(1)
            # for all other tags, a 0
            for tag in(vtags - set([EMPTY_TAG, pred_tag])):
                pred_ys_by_tag[tag].append(0)
        del pred_ys_by_tag[EMPTY_TAG]
    return pred_ys_by_tag

def split_dict(dct, train_split):
    td, vd = {}, {}
    for key, lst in dct.items():
        td[key] = lst[:train_split]
        vd[key] = lst[train_split:]
    return td, vd

## Load and Pre-Process Essays

In [3]:
%%time
from BrattEssay import load_bratt_essays
from load_data import load_process_essays
from collections import defaultdict
from IterableFP import flatten
from Settings import Settings

settings = Settings()
root_folder = settings.data_directory + "CoralBleaching/Thesis_Dataset/"
training_folder = root_folder + "Training" + "/"
config = get_config(training_folder)

tagged_essays = load_process_essays(**config)

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/
902 files found
902 essays processed
CPU times: user 48.3 s, sys: 280 ms, total: 48.6 s
Wall time: 49.8 s


In [4]:
print("Started at: " + str(datetime.datetime.now()))
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger()

TEST_SPLIT          = 0.2

Started at: 2017-03-05 10:45:16.476211


In [6]:
from numpy.random import shuffle
shuffle(tagged_essays)

## Prepare Tags

In [7]:
tag_freq = defaultdict(int)
for essay in tagged_essays:
    for sentence in essay.sentences:
        for word, tags in sentence:
            for tag in tags:
                tag_freq[tag] += 1

In [9]:
EMPTY_TAG = "Empty"
regular_tags = list((t for t in tag_freq.keys() if t[0].isdigit()))
vtags = set(regular_tags)
vtags.add(EMPTY_TAG)
sorted(vtags)

['1',
 '11',
 '12',
 '13',
 '14',
 '2',
 '3',
 '4',
 '5',
 '50',
 '5b',
 '6',
 '7',
 'Empty']

# Transform Essays into Training Data (Word Ids)

In [10]:
from collections import defaultdict
generator = idGen(seed=1) # important as we zero pad sequences

ix2tag = {}
for ix, t in enumerate(vtags):
    ix2tag[ix] = t

def ids2tags(ids):
    return [generator.get_key(j) for j in ids]  

def lbls2tags(ixs):
    return [ix2tag[ix] for ix in ixs]

xs = []
ys = []

START = "<start>"
END   = "<end>"

# cut texts after this number of words (among top max_features most common words)
maxlen = 0
ys_bytag = defaultdict(list)
seq_lens = []
for essay in tagged_essays:
    for sentence in essay.sentences:
        row = []
        y_found = False
        y_seq = []
        for word, tags in [(START, set())] + sentence + [(END, set())]:
            id = generator.get_id(word) #starts at 0, but 0 used to pad sequences
            row.append(id)
            
            tags = vtags.intersection(tags)
            if word != START and word != END:
                for t in (vtags - set([EMPTY_TAG])):
                    if t in tags:
                        ys_bytag[t].append(1)
                    else:
                        ys_bytag[t].append(0)
            
            if len(tags) > 1:
                most_common = max(tags, key=lambda t: tag_freq[t])
                tags = set([most_common])
            if len(tags) == 0:
                tags.add(EMPTY_TAG)
            
            one_hot = []
            for t in vtags:
                if t in tags:
                    one_hot.append(1)
                else:
                    one_hot.append(0)
            y_seq.append(one_hot)
    
        seq_lens.append(len(row)-2)
        ys.append(y_seq)
        xs.append(row)
        maxlen = max(len(row), maxlen)

max_features=len(generator.get_ids())+1 #Need plus one maybe due to masking of sequences

## Create Train - Test Split

In [11]:
maxlen = maxlen
num_training = int((1.0 - TEST_SPLIT) * len(xs))

X_train_orig, y_train_orig, X_test_orig, y_test_orig = xs[:num_training], ys[:num_training], xs[num_training:], ys[num_training:]
seq_len_train, seq_len_test = seq_lens[:num_training], seq_lens[num_training:]

num_train_wds = sum(seq_len_train)
num_wds       = sum(seq_lens)
train_ys_bytag, test_ys_by_tag = split_dict(ys_bytag, num_train_wds)

### Validate Dictionary Splitting Code

In [13]:
code = '50'
# Values below should align
print(num_train_wds, num_wds - num_train_wds, num_wds)
print(len(train_ys_bytag[code]),len(test_ys_by_tag[code]),len(ys_bytag[code]))
a = num_train_wds, num_wds - num_train_wds, num_wds
b = len(train_ys_bytag[code]),len(test_ys_by_tag[code]),len(ys_bytag[code])
assert a==b, "Splitting logic does not work correctly"

109742 27424 137166
109742 27424 137166


In [14]:
X_train = sequence.pad_sequences(X_train_orig, maxlen=maxlen) #30 seems good
X_test  = sequence.pad_sequences(X_test_orig,  maxlen=maxlen)

y_train = sequence.pad_sequences(y_train_orig, maxlen=maxlen)
y_test  = sequence.pad_sequences(y_test_orig,  maxlen=maxlen)

print('X_train shape:', X_train.shape)
print('X_test shape: ',  X_test.shape)
print()
print('y_train shape:', y_train.shape)
print('y_test shape: ',  y_test.shape)

X_train shape: (6633, 93)
X_test shape:  (1659, 93)

y_train shape: (6633, 93, 14)
y_test shape:  (1659, 93, 14)


In [15]:
embedding_size = 64
hidden_size    = 64
out_size = len(vtags)

model = Sequential()
model.add(Embedding(max_features, embedding_size, input_length=maxlen, mask_zero=True))
model.add(LSTM(hidden_size, return_sequences=True))  
#merge
model.add(TimeDistributedDense(out_size))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['fmeasure'], sample_weight_mode="temporal")

/Users/simon.hughes/anaconda3/envs/tensorflow_cpu/lib/python3.5/site-packages/keras/layers/core.py:1206: UserWarning: `TimeDistributedDense` is deprecated, And will be removed on May 1st, 2017. Please use a `Dense` layer instead.
  warnings.warn('`TimeDistributedDense` is deprecated, '


In [ ]:
batch_size = 128
epochs = 6

results = model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=epochs, validation_split=0.2, verbose=1)
probs = model.predict_proba(X_test, batch_size=batch_size)   
test_pred_ys_by_tag = collapse_results(seq_len_test, probs)
class2metrics = ResultsProcessor.compute_metrics(test_ys_by_tag, test_pred_ys_by_tag)
micro_metrics = micro_rpfa(class2metrics.values())
micro_metrics

Train on 5306 samples, validate on 1327 samples
Epoch 1/6
4480/5306 [========================>.....] - ETA: 2s - loss: 0.7059 - fmeasure: 0.8189 

In [25]:
code = "50"
assert len(test_pred_ys_by_tag[code]) == sum(seq_len_test) == len(test_ys_by_tag[code])
len(test_pred_ys_by_tag[code]), sum(seq_len_test), len(test_ys_by_tag[code])

(27424, 27424, 27424)

### TODO
* Look into masking
* Try Bi-Directional LSTM (see example below)
* Use early stopping criteria

In [ ]:
""" Bi Directional LSTM example from - https://github.com/fchollet/keras/issues/3086

from keras.models import Model
import numpy as np
from keras.layers import Masking, Activation, Input, LSTM, merge
a = np.array([[[.3,.1,.2,.2,.1,.1],[.2,.3,.3,.3,.3,.1],[0,0,0,0,0,0]]])

inputs = Input(shape=(3,6))
mask = Masking(mask_value=0.0)(inputs)
fw = LSTM(1,return_sequences=True)(mask)
bw = LSTM(1,return_sequences=True,go_backwards=True)(mask)
merged = merge([fw,bw],mode='sum')
model = Model(input=inputs,output=fw)
model2 = Model(input=inputs,output=bw)
model3 = Model(input=inputs,output=merged)
"""